In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# Problem Setup

In [ ]:
from trajectory import Trajectory
from environment import Environment
from global_variables import DIM

n_anchors = 5 #number of anchors
n_positions = 10 #number of robot sample positions
n_complexity = 4 #model complexity

trajectory = Trajectory(n_positions, n_complexity)
environment = Environment(n_anchors)

trajectory.set_trajectory(seed=1)
environment.set_random_anchors(seed=1)


plt.figure()
environment.plot()
trajectory.plot(color='orange')
#plt.axis('off')
#plt.savefig('traj_setup.png')

environment.set_D(trajectory)
D_topright = environment.D[:n_positions,n_positions:] 

## SDP - based approach
### Noiseless case - single experiment

In [ ]:
print("Make sure that your cvxpy version is >= 1.0.6!")
import cvxpy
print("Your version:", cvxpy.__version__)

from solvers import OPTIONS, semidefRelaxationNoiseless

# We cane change the global variable OPTIONS here. 

#OPTIONS[cvxpy.SCS]["max_iters"] = 200
# Seems to have no effect: 
#OPTIONS[cvxpy.SCS]["use_indirect"] = False 
# Seems to have no effect either: 
#OPTIONS[cvxpy.SCS]["eps"] = 1e-1
# Seems to have no effect either: 
#OPTIONS[cvxpy.SCS]["scale"] = 1

# Fails completely without this:
#OPTIONS[cvxpy.CVXOPT]["kktsolver"] = "robust"

# have no effect:
#OPTIONS[cvxpy.CVXOPT]["feastol"] = 1e-3
#OPTIONS[cvxpy.CVXOPT]["reltol"] = 1e-5
#OPTIONS[cvxpy.CVXOPT]["abstol"] = 1e-5

# leads to faster non-convergence: 
#OPTIONS[cvxpy.CVXOPT]["refinement"] = 0

#OPTIONS[cvxpy.SCS]["verbose"] = False
print(trajectory.basis.shape)

X = semidefRelaxationNoiseless(D_topright, environment.anchors, trajectory.basis, chosen_solver=cvxpy.CVXOPT)
print('should be identity:\n', X[:DIM, :DIM])
print('should be equal:\n', X[:DIM:, DIM:])
print(trajectory.coeffs)

In [ ]:
def robust_increment(arr, i, j):
    """ increment value of array if inside bound, and set to 1 if previously nan. """
    if j < arr.shape[1]:
        # TODO nan test not required anymore. 
        if np.isnan(arr[i, j]):
            arr[i, j] = 1
        else:
            arr[i, j] += 1
    

# multiple setups. 
from solvers import OPTIONS, semidefRelaxationNoiseless
environment = Environment(n_anchors)


n_complexity = 4
n_anchors = 7
positions = np.arange(1, 10)

successes = np.full((len(positions), len(positions) * n_anchors), np.nan)

num_not_solved = np.full(successes.shape, np.nan)
num_not_accurate = np.full(successes.shape, np.nan)

n_its = 5

for n_it in range(n_its):
    for i, n_positions in enumerate(positions):
        print('n_positions', n_positions)
        trajectory = Trajectory(n_positions, n_complexity)

        trajectory.set_trajectory(seed=n_it)
        environment.set_random_anchors(seed=n_it)
        environment.set_D(trajectory)
        
        # remove some measurements
        
        n_measurements = n_positions * n_anchors
        
        pairs = np.array(np.meshgrid(range(n_positions), range(n_anchors)))
        pairs.resize((2, n_positions * n_anchors))
        for j, n_missing in enumerate(range(n_measurements)):
            
            # set all values to 0 since we have visited them. 
            if np.isnan(successes[i, j]):
                successes[i, j] = 0.0 
            if np.isnan(num_not_solved[i, j]):
                num_not_solved[i, j] = 0.0 
            if np.isnan(num_not_accurate[i, j]):
                num_not_accurate[i, j] = 0.0
            
            #print('n_misisng', n_missing)
            D_topright = environment.D[:n_positions,n_positions:].copy()
            indices = np.random.choice(n_measurements, size=n_missing, replace=False)
            xs = pairs[0, indices]
            ys = pairs[1, indices] 
            assert len(xs) == n_missing
            assert len(ys) == n_missing
            D_topright[xs, ys] = 0.0
            
            # assert correct number of missing measurements
            idx = np.where(D_topright == 0.0)
            assert n_missing == len(idx[0])
            
            try:
                X = semidefRelaxationNoiseless(D_topright, environment.anchors, trajectory.basis, chosen_solver=cvxpy.CVXOPT)
                
                assert not np.any(np.abs(X[:DIM, DIM:] - trajectory.coeffs) > 1e-10) 
                
                # TODO: why does this not work? 
                #assert np.testing.assert_array_almost_equal(X[:DIM, DIM:], trajectory.coeffs)
                
                robust_increment(successes, i, j)
            except cvxpy.SolverError:
                #print("could not solve n_positions={}, n_missing={}".format(n_positions, n_missing))
                robust_increment(num_not_solved, i, j)
            except ZeroDivisionError:
                #print("could not solve n_positions={}, n_missing={}".format(n_positions, n_missing))
                robust_increment(num_not_solved, i, j)
            except AssertionError:
                #print("result not accurate n_positions={}, n_missing={}".format(n_positions, n_missing))
                robust_increment(num_not_accurate, i, j)


In [ ]:
import time
import os
from experiment import save_params, save_results

key = 'fixed_complexity'

complexities = [n_complexity]
anchors = [n_anchors]

parameters = {
    'key': key, 
    'time': time.time(), 
    'positions': positions, 
    'complexities': complexities, 
    'anchors': anchors, 
}

results = {
    'successes': successes, 
    'num-not-solved': num_not_solved, 
    'num-not-accurate': num_not_accurate}

if not os.lstat('./results'):
    os.mkdir('./results')
    
foldername = 'results/{}/'.format(key)
if not os.lstat(foldername):
    os.mkdir(foldername)

save_params(foldername + 'parameters.json', **parameters)
save_results(foldername + 'result_{}_{}.csv', results)

In [ ]:
## Experiment 1: fixed K, increasing N and increasing number of missing measurements
from experiment import read_results
foldername = 'results/{}/'.format(key)

#print(foldername.split('/')[:-1])

def read_results(filestart):
    results = {} 
    dirname = os.path.dirname(filestart)
    for filename in os.listdir(dirname):
        full_path = os.path.join(dirname, filename)
        if os.path.isfile(full_path) and filestart in full_path:
            print('reading', full_path)
            key = filename.split('_')[-2]
            new_array = np.loadtxt(full_path, delimiter=',')
            if key in results.keys():
                results[key] += new_array
            else:
                print('new key:', key)
                results[key] = new_array
    return results

results = read_results(foldername + 'result_')

In [ ]:
def add_plot_decoration():
    plt.colorbar()
    plt.xlabel('number of missing measurements')
    plt.ylabel('n_positions')
    plt.yticks(range(len(positions)), positions)
    plt.gca().xaxis.tick_bottom()
    plt.show()
    
print("NOTE: values are nan if never visited (white)")
    
plt.matshow(results['successes'])
plt.title('Number of successful solutions')
add_plot_decoration()


plt.matshow(results['num-not-solved'])
plt.title('Number of solver failures')
add_plot_decoration()

plt.matshow(results['num-not-accurate'])
plt.title('Number of inaccurate solutions')
add_plot_decoration()

bound_theory = np.zeros_like(successes)
n_min = n_complexity * (n_complexity + 1) / 2 + DIM * n_complexity
print('needed number of measurements:',  n_min)
for i, n_pos in enumerate(positions):
    max_missing_allowed = max(int(n_pos * n_anchors - n_min), 0)
    if max_missing_allowed < bound_theory.shape[1]:
        bound_theory[i, max_missing_allowed] = 1.0
    if n_pos * n_anchors < bound_theory.shape[1]:
        bound_theory[i, (n_pos * n_anchors):] = np.nan

print("NOTE: shows theoretical bound () and value from which zero measurements are left (white)")
plt.matshow(bound_theory)
plt.title('theoretical bound')
add_plot_decoration()

In [ ]:
from solvers import semidefRelaxation
from solvers import OPTIONS
#OPTIONS[cvxpy.SCS]["max_iters"] = 00
X = semidefRelaxation(D_topright, environment.anchors, trajectory.basis)

plt.matshow(X[:20, :20])
plt.colorbar()

plt.matshow(X[-20:, -20:])
plt.colorbar()

print('should be identity:\n', X[:DIM, :DIM])
print('should be equal:\n', X[DIM:, DIM:])
print(coeffs)

## Investigate null space

In [ ]:
#Write all constraints as a matrix vector product

from constraints import *

ConstraintsMat, ConstraintsVec = get_constraints_matrix(D_topright, environment.anchors, trajectory.basis)
ConstraintsMat=np.array(ConstraintsMat)
ConstraintsVec=np.array(ConstraintsVec)

print(np.isclose(ConstraintsMat@trajectory.Z_opt.flatten(),ConstraintsVec))

print(ConstraintsMat.shape)
print(ConstraintsVec.shape)
u, s, vh = np.linalg.svd(ConstraintsMat, full_matrices=True)
print(np.around(s,3))

In [ ]:
#construct right inverse and check meets constraints
num_zero_SVs = len(np.where(s<1e-10)[0])
Z_hat = vh[:-num_zero_SVs,:].T@np.diag(1/s[:-num_zero_SVs])@u[:,:len(s)-num_zero_SVs].T@ConstraintsVec #right inverse
Z_hat = Z_hat.reshape([DIM + n_complexity,DIM + n_complexity])
print(np.isclose(ConstraintsMat@Z_hat.flatten(),ConstraintsVec)) #should satisfy constraints since it's a right inverse
coeffs_hat = Z_hat[:DIM,DIM:]
print(np.isclose(trajectory.coeffs,coeffs_hat))

In [ ]:
print('find basis vectors of null space')
tmp = vh[-num_zero_SVs:,:]
print(tmp.shape)
nullSpace = []
for i in range(num_zero_SVs):
    nullSpace.append(tmp[i,:].reshape([DIM + n_complexity,DIM + n_complexity]))

nullSpace = np.array(nullSpace)
Z_hat2 = Z_hat+nullSpace[0,:]+2*nullSpace[1,:]+3*nullSpace[2,:]
print(np.isclose(ConstraintsMat@(Z_hat2.flatten()),ConstraintsVec))

In [ ]:
print(np.around(nullSpace[0,:],5))
print(np.around(nullSpace[1,:],5))
print(np.around(nullSpace[2,:],5))

### Solve as linear program

In [ ]:
# try linear program
from cvxopt import matrix, solvers

tmpMat=np.diag(s[:-num_zero_SVs])@vh[:-num_zero_SVs,:]
tmpVec = u[:,:len(s)-num_zero_SVs].T@ConstraintsVec
A = matrix(np.vstack([tmpMat,-tmpMat]))
b = matrix(np.hstack([tmpVec,-tmpVec]))
c = matrix(np.ones((DIM + n_complexity)*(DIM + n_complexity)))
sol=solvers.lp(c,A,b, solver='glpk') #doesn't work eith defualt solver but glpk good
print(sol)
Z_hat = np.array(sol['x']).reshape([DIM + n_complexity,DIM + n_complexity])
print(np.isclose(Z_hat,trajectory.Z_opt))

### Debugging - End

## MDS - based approach
### Noiseless case

In [ ]:
# find new coefficients
from solvers import customMDS
coeffs_est = customMDS(D_topright, trajectory.basis, environment.anchors)
print(coeffs_est.shape)
print(coeffs_est)
print(trajectory.coeffs)

In [ ]:
from trajectory import Trajectory
trajectory_est = Trajectory(n_positions, n_complexity)
trajectory_est.set_trajectory(coeffs=coeffs_est)

plt.figure()
environment.plot()
trajectory_est.plot(color='red', linestyle=':')
trajectory.plot(color='green', linestyle='--')

### Noisy case

In [ ]:
from solvers import gradientDescent

sigma = 4

np.random.seed(1)
D_topright_noisy = D_topright + sigma * np.random.randn(*(D_topright.shape))
coeffs_est_noisy = customMDS(D_topright_noisy, trajectory.basis, environment.anchors)

coeffs_est_noisy, costs = gradientDescent(
    environment.anchors, trajectory.basis, coeffs_est_noisy, 
    D_topright_noisy,maxIters=50)
#print(checkStationaryPointSRLS(A,F,C_hat,DTR_tilde))
#plt.plot(costs)

In [ ]:
trajectory_est_noisy = Trajectory(n_positions, n_complexity)
trajectory_est_noisy.set_trajectory(coeffs=coeffs_est_noisy)

plt.figure()
environment.plot()
trajectory_est_noisy.plot(color='red', linestyle=':')
trajectory.plot(color='green', linestyle='--')

### Missing measurements

missing measurements between anchors, and between anchors and robot

In [ ]:
from solvers import alternateGDandKEonDR

np.random.seed(1)

sigma = 2
missing_proportion = 0.8

D_right = environment.D[:,n_positions:]
mask = np.ones(D_right.shape)

mask[:n_positions, :] *= (np.random.rand(n_positions,n_anchors)>missing_proportion)
D_right_missing = mask * D_right

# TODO: why are anchor positions not noisy? 
np.random.seed(1)
D_right_missing[:n_positions,:] += sigma * np.random.randn(n_positions, n_anchors)
D_right_est, errs = alternateGDandKEonDR(D_right_missing, mask, trajectory.basis, environment.anchors, 
                                         niter=40, DR_true=D_right)
plt.plot(errs)
coeffs_est_missing = customMDS(D_right_est[:n_positions,:], trajectory.basis, environment.anchors)
#print(C_hat)
#print(C)

In [ ]:
trajectory_est_missing = Trajectory(n_positions, n_complexity)
trajectory_est_missing.set_trajectory(coeffs=coeffs_est_missing)

plt.figure()
trajectory_est_missing.plot(color='red')
trajectory.plot(color='green')
environment.plot()

In [ ]:
plt.figure()
trajectory_est_missing.plot(color='red', mask=mask)
trajectory.plot(color='green')
environment.plot()
#plt.axis('off')
#plt.savefig('traj_useful.png')

In [ ]:
trajectory_est_missing.plot_number_measurements(mask=mask)

In [ ]:
D_right_est, errs = alternateGDandKEonDR(D_right_missing, 
                                    mask, trajectory.basis, environment.anchors, 
                                    niter=30, DR_true=D_right)
plt.plot(errs)
coeffs_est = customMDS(D_right_est[:n_positions,:], trajectory.basis, environment.anchors)

In [ ]:
trajectory_est = Trajectory(n_positions, n_complexity)
trajectory_est.set_trajectory(coeffs=coeffs_est)

plt.figure()
trajectory_est.plot(color='red')
trajectory.plot(color='green')

In [ ]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})
tmp = D_right[:n_positions,:]
print(tmp[mask[:n_positions,:]==1])
print(mask[:n_positions,:]*tmp)

tmp = D_right_missing[:n_positions,:]
print(tmp[mask[:n_positions,:]==1])
print(mask[:n_positions,:]*tmp)